In [5]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import netcomp as nc
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import mean_squared_error
import pycombo
# from netAPI import visualizePartitionShape
# from netAPI import getComboPartition
# from netAPI import getComboSeries
import geopandas as gpd
import matplotlib.pyplot as plt

## Community detection

In [6]:
def makeGraphfromDf(df,oCol, dCol, weight):
    G=nx.DiGraph()
    nx.set_edge_attributes(G,'weight', 0)
    for k in df.index:
        G.add_edge(df[oCol][k],df[dCol][k],weight=df[weight][k])
#     nx.write_edgelist(G, comboPath+'temp/%s.net'%city)
    return G

In [7]:
def reClass(commDict):
    label, index = np.unique(np.fromiter(commDict.values(),dtype=int), return_index=True)
    recommDict = {}
    for zone in commDict.keys():
        recommDict[zone] = recommDict[commDict[zone]]
    return reClass

In [8]:
def fixedARI(group1, group2):
    from sklearn.metrics import pair_confusion_matrix

    (tn, fp), (fn, tp) = pair_confusion_matrix(group1, group2)
    (tn, fp), (fn, tp) = (float(tn), float(fp)), (float(fn), float(tp))
    if fn == 0 and fp == 0:
        return 1.0

    return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
                                           (tp + fp) * (fp + tn))

In [21]:
def weightedAdjustedRandIndex(df,oCol, dCol,groundTruthCol, predCol,maxcom):
    GTruth = makeGraphfromDf(df,oCol, dCol, groundTruthCol)
    GPred = makeGraphfromDf(df,oCol, dCol, predCol)
    population = df.groupby(['origin']).agg({'flowReal':'sum'}).reset_index()

    population['flowReal'] = population['flowReal'].astype(int)
    groundTruthComm,groundTruthMod = pycombo.execute(GTruth, max_communities = maxcom)
    
    predictComm,predictMod = pycombo.execute(GPred, max_communities = maxcom)
    weight = dict(zip(population.origin, population.flowReal))
#     print(population)
    groundTruthCommWeighted = []
    predictCommWeighted = []
    for zone in sorted(groundTruthComm.keys()):
        if zone in weight:
            groundTruthCommWeighted += [groundTruthComm[zone]]*weight[zone]
            predictCommWeighted += [predictComm[zone]]*weight[zone]

        else:
            pass
    groundTruthCommWeighted = np.array(groundTruthCommWeighted, dtype=np.float64)
    predictCommWeighted = np.array(predictCommWeighted, dtype=np.float64)
#     ri = rand_score(groundTruthCommWeighted,predictCommWeighted)
    mse = mean_squared_error(df[groundTruthCol],df[predCol])
    ari = fixedARI(groundTruthCommWeighted,predictCommWeighted)
    nmi = normalized_mutual_info_score(groundTruthCommWeighted,predictCommWeighted)
#     countCommGT,countCommPr = len(set(groundTruthComm.values())),len(set(predictComm.values()))
#     return (ri,ari,nmi,countCommGT,countCommPr)
    return(mse,ari,nmi)

In [22]:
def plotMetrics(cities,resultList,modelList,metricLoc=0):
    modelAmount = len(modelList)
    metric = {0:'RI',1:'ARI',2:'NMI'}
    metricName = metric[metricLoc]

    x = np.arange(len(cities))  # the label locations
    width = 0.2  # the width of the bars
    
    if modelAmount % 2 == 0:
        shift = width*0.5
    else:
        shift = 0
    
    fig, ax = plt.subplots(figsize=(16,8))
    
    for i in range(modelAmount):
        plt.bar(x-(modelAmount-1)/2*width+i*width,
                np.array(list(resultList[i].values()))[:,metricLoc],width=width,
                label=modelList[i])


   
    # plt.title('Adjusted rand score')

   
    
    _ = plt.xticks(ticks=x,labels=cities,rotation=70)
    plt.legend(loc='best',bbox_to_anchor=(0.5, 0., 0.5, 0.5))
#     plt.title('%s comparison between noconstrain and doubly constrain gravity model for income groups'%metricName)

In [23]:
cities = ['New York City',
    'Los Angeles',
    'Chicago',
    'Houston',
    'Boston',
    'Phoenix',
    'Philadelphia',
    'San Antonio',
    'San Diego', 
    'Dallas', 
    'San Jose', 
    'Austin']

In [26]:
seed = 2017
np.random.seed(seed)
NoconstrainPowerLaw = {}
NoconstrainFullPowerLaw = {}
NoconstrainExp = {}
constrainDist = {}
classicConstrainDist = {}
# cities = ['New York City', 'Los Angeles', 'Chicago', 'Houston', 'Boston', 'Phoenix', 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin']
for city in cities:
    print(city)
    total = pd.read_csv('../unconstrainCTPowerlaw/%s.csv'%city)
    total['flowPred'] = total['S000pred']
    total['flowReal'] = total['S000flow']
    total = total.groupby(['origin','destination']).sum().reset_index()
    mse,ari,nmi = weightedAdjustedRandIndex(total,'origin','destination','S000flow','S000pred',0)
    NoconstrainPowerLaw[city] = [mse,ari,nmi]

    total = pd.read_csv('../unconstrainCTFullPowerlaw/%s.csv'%city)
    total['flowPred'] = total['S000pred']
    total['flowReal'] = total['S000flow']
    total = total.groupby(['origin','destination']).sum().reset_index()
    mse,ari,nmi = weightedAdjustedRandIndex(total,'origin','destination','S000flow','S000pred',0)
    NoconstrainFullPowerLaw[city] = [mse,ari,nmi]
    
    total = pd.read_csv('../constrainCTdistbinsAB/%s.csv'%city)
    total['flowPred'] = total['S000pred']
    total['flowReal'] = total['S000flow']
    total = total.groupby(['origin','destination']).sum().reset_index()
    mse,ari,nmi = weightedAdjustedRandIndex(total,'origin','destination','S000flow','S000pred',0)
    classicConstrainDist[city] = [mse,ari,nmi]

New York City
Los Angeles
Chicago
Houston
Boston
Phoenix
Philadelphia
San Antonio
San Diego
Dallas
San Jose
Austin


In [29]:
## df = pd.DataFrame(index=cities)
df['NoconstrainPowerLaw'] = np.array(list(NoconstrainPowerLaw.values()))[:,2]
# df['A2'] = np.array(list(NoconstrainExp.values()))[:,0]
df['NoconstrainFullPowerLaw'] = np.array(list(NoconstrainFullPowerLaw.values()))[:,2]
df['DoublyConstrain'] = np.array(list(classicConstrainDist.values()))[:,2]
df

,NoconstrainPowerLaw,NoconstrainFullPowerLaw,DoublyConstrain
New York City,0.717796,0.710576,0.714424
Los Angeles,0.736189,0.878277,0.724127
Chicago,0.564453,0.581617,0.564815
Houston,0.624485,0.621707,0.551690
Boston,0.559127,0.650125,0.676648
Phoenix,0.541096,0.535273,0.665939
Philadelphia,0.535641,0.530895,0.488246
San Antonio,0.737338,0.740900,0.734498
San Diego,0.530143,0.527629,0.628607
Dallas,0.523532,0.535669,0.523089


In [21]:
df = pd.DataFrame(index=cities)
df['NoconstrainPowerLaw'] = np.array(list(NoconstrainPowerLaw.values()))[:,0]
# df['A2'] = np.array(list(NoconstrainExp.values()))[:,0]
df['NoconstrainFullPowerLaw'] = np.array(list(NoconstrainFullPowerLaw.values()))[:,0]
df['DoublyConstrain'] = np.array(list(classicConstrainDist.values()))[:,0]
df